# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

# 對資料做合併、取出
train_Y = df_train['Survived']                                  # 取出目標值
ids = df_test['PassengerId']                                    # 取出 ID
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)  # 去掉 目標值、ID
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])                              # 合併資料
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':                           # 篩選出類別型欄位
        object_features.append(feature)             # 將欄位名字加入 list
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]                            # 留下類別型欄位
df = df.fillna('None')                              # 填補空缺值NAN 為 'None'
train_num = train_Y.shape[0]                        # 獲得train資料數
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [4]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
start = time.time()
le = LabelEncoder()
df_temp = copy.deepcopy(df)                                     # 深度複製

# 進行標籤編碼
for col in df.columns:
    df_temp[col] = le.fit_transform( df[col] )                  # 做 LabelEncoding 
train_X = df_temp[:train_num]                                   # 取出 train 資料

# 進行回歸分數計算
estimator = LogisticRegression()
score = cross_val_score(estimator, train_X, train_Y ).mean()    # 計算分數

# 印出
print(f'time : { time.time() - start }')
print(f'score : { score }')
print(f'train_X_Shape : {train_X.shape}')

time : 0.17353391647338867
score : 0.7800138095537004
train_X_Shape : (891, 5)


In [5]:
# 均值編碼 + 邏輯斯迴歸
start = time.time()
data = pd.concat( [df[:train_num],train_Y], axis=1)                     # 合併 train 資料和目標值

# 進行均值編碼
for col in df.columns:
    mean_df = data.groupby([col])['Survived'].mean().reset_index()      # 每個欄位 做分類並計算其各類別['Survived']平均值
    mean_df.columns = [ col, f'{col}_mean']                             # 重設欄位名稱
    data = pd.merge( data, mean_df, on=col, how='left')                 # 合併全部資料和[分類平均]  // on=col 利用此欄做合併 // how='left' 如左右df有不同，保留左邊資料 //
    data.drop([col], axis=1, inplace=True)                              # 去掉原欄位
data.drop(['Survived'], axis=1, inplace=True )                          # 去掉 ['Survived'] 欄位                 

# 進行回歸分數計算
estimator = LogisticRegression()
score = cross_val_score(estimator, data, train_Y).mean()                # 計算分數

# 印出
print(f'time : {time.time()- start}')
print(f'score : {score}')                                               # 計算分數 // 分數 = 1 有可能是 overfitting
print(f'train_X_Shape : {train_X.shape}')

time : 0.09374761581420898
score : 1.0
train_X_Shape : (891, 5)


## 因為有些欄位類別數量太多，會造成 overfitting(如上)
## 所以可以剃除那些類別太多的

In [12]:
# 每欄類別數量
print( df.nunique() )

# 會發現 Name、Ticket 類別數特別多
# 加以剃除
df_temp = df.drop( ['Name','Ticket'], axis=1 )
print('\n\n', df_temp.nunique() )

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64


 Sex           2
Cabin       187
Embarked      4
dtype: int64


### 利用更動後資料，重新計算

In [13]:
 # 均值編碼 + 邏輯斯迴歸
start = time.time()
data = pd.concat( [df_temp[:train_num],train_Y], axis=1)                     # 合併 train 資料和目標值

# 進行均值編碼
for col in df_temp.columns:
    mean_df = data.groupby([col])['Survived'].mean().reset_index()      # 每個欄位 做分類並計算其各類別['Survived']平均值
    mean_df.columns = [ col, f'{col}_mean']                             # 重設欄位名稱
    data = pd.merge( data, mean_df, on=col, how='left')                 # 合併全部資料和[分類平均]  // on=col 利用此欄做合併 // how='left' 如左右df有不同，保留左邊資料 //
    data.drop([col], axis=1, inplace=True)                              # 去掉原欄位
data.drop(['Survived'], axis=1, inplace=True )                          # 去掉 ['Survived'] 欄位                 

# 進行回歸分數計算
estimator = LogisticRegression()
score = cross_val_score(estimator, data, train_Y).mean()                # 計算分數

# 印出
print(f'time : {time.time()- start}')
print(f'score : {score}')                                               # 計算分數 // 分數 = 1 有可能是 overfitting
print(f'train_X_Shape : {train_X.shape}')

time : 0.05983781814575195
score : 0.835019772770071
train_X_Shape : (891, 5)


### 可以發現分數比[ 標籤編碼 ]高，也不會 overfitting 